<a href="https://colab.research.google.com/github/thomasbad/AI_Model_PICw_GoogleColab/blob/main/sd_lora_webui_colab_mka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SD+LoRA WebUI Colab MKA NSFW Edition

### How to Use

1. Go to **Setup model** and **Advanced Model** to do the customization
2. On the tool bar, go to "Runtime" > "Change runtime type", and change it to GPU
3. Select "Runtime" > "Run all"
4. Go to **Startup WebUI** and click public URL to do your stuff

(P.S. This thing will use your Google Drive storage to store the downloaded model for use, ensure you have enough space to store them, and remove them if you don't want them take up your space)

### Reference
https://github.com/ddPn08/automatic1111-colab

## 1 - Pre-configuration

In [5]:
# @title 1.1 Download stable-diffusion-webui
%cd /content/
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "master"  # @param {type: "string"}

! git clone {repository_url}
%cd /content/stable-diffusion-webui
! git checkout {webui_branch}

/content
fatal: destination path 'stable-diffusion-webui' already exists and is not an empty directory.
/content/stable-diffusion-webui
D	embeddings/Place Textual Inversion embeddings here.txt
D	models/Stable-diffusion/Put Stable Diffusion checkpoints here.txt
D	models/VAE/Put VAE here.txt
Already on 'master'
Your branch is up to date with 'origin/master'.


In [6]:
# @title 1.2 Create Cloud folder
%cd /content
# @markdown Cloud folder name
data_dir_name = "Stable_Diffusion_WebUI_Colab"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)

for script in os.listdir(scripts_path):
    ! rm -Rf stable-diffusion-webui/scripts/{script} && ln -s {scripts_path}/{script} stable-diffusion-webui/scripts/{script}

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf stable-diffusion-webui/embeddings && ln -s {models_path}/embeddings stable-diffusion-webui/embeddings
    else:
        ! rm -Rf stable-diffusion-webui/models/{dir} && ln -s {models_path}/{dir} stable-diffusion-webui/models/{dir}

! rm -Rf stable-diffusion-webui/outputs && ln -s {data_dir}/outputs stable-diffusion-webui/outputs

for filename in ["config.json", "ui-config.json", "styles.csv", "artists.csv"]:
    ! rm -f stable-diffusion-webui/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    if filename == "styles.csv":
        ! cp {config_path}/styles.csv stable-diffusion-webui/styles.csv
    else:
        ! ln -s {config_path}/{filename} stable-diffusion-webui/{filename}

/content
Mounted at /content/drive


## 2 - Setup model

In [7]:
# @title 2.1 Setup SD Model
# @markdown Do you need to download the SD model AIO package?
need_to_download_sd = True # @param {type:"boolean"}
# @markdown Select the SD model AIO package for use :
model_package = "AnythingV4.5" # @param ["AnythingV3", "AnythingV4.5", "AbyssOrangeMix2_sfw", "AbyssOrangeMix3", "ChilloutMix", "Others(Input below)"] 
# @markdown URL of other SD model :
sd_url = "" #@param {type:"string"}
vae_url = "" #@param {type:"string"}
# @markdown ---
# @markdown - The pictures' colors will be more mich after VAE is enabled with:<br>
# @markdown  - WebUI > Settings > Stable Diffusion > SD VAE Setting<br>
# @markdown  - Remember to click "Apply settings" for change

sd_urls = []
vae_urls = []
if model_package == "AnythingV3":
    sd_urls.append("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/my_vae/resolve/main/Anything-V3.0.vae.pt")
elif model_package == "AnythingV4.5":
    sd_urls.append("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors")
    vae_urls.append("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt")
elif model_package == "AbyssOrangeMix2_sfw":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_sfw.safetensors")
    vae_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt")
elif model_package == "AbyssOrangeMix3":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3.safetensors")
    vae_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt")
elif model_package == "ChilloutMix":
    sd_urls.append("https://huggingface.co/WuLing/Chilloutmix_package/resolve/main/chilloutmix_NiPrunedFp32.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/Chilloutmix_package/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")
elif model_package == "Others(Input below)":
    sd_urls.append(sd_url)
    vae_urls.append(vae_url)

if sd_urls != [] and need_to_download_sd == True:
    %cd {models_path}/Stable-diffusion
    for sd_url in sd_urls:
        if sd_url != "":
            ! wget -nc --content-disposition {sd_url}

if vae_urls != [] and need_to_download_sd == True:
    %cd {models_path}/VAE
    for vae_url in vae_urls:
        if vae_url != "":  
            ! wget -nc --content-disposition {vae_url}

/content/drive/MyDrive/Stable_Diffusion_WebUI_Colab/models/Stable-diffusion
--2023-03-10 04:08:23--  https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ba/33/ba33ee3f7d75269b9442805ec46dc9e9e7093cd256c781ba470f5b4d6481656a/6e430eb51421ce5bf18f04e2dbe90b2cad437311948be4ef8c33658a73c86b2a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27anything-v4.5-pruned.safetensors%3B+filename%3D%22anything-v4.5-pruned.safetensors%22%3B&Expires=1678679457&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2JhLzMzL2JhMzNlZTNmN2Q3NTI2OWI5NDQyODA1ZWM0NmRjOWU5ZTcwOTNjZDI1NmM3ODFiYTQ3MGY1YjRkNjQ4MTY1NmEvNmU0MzBlYjUxNDI

In [8]:
# @title 2.2 Setup Embedding + Hypernetwork + LoRA
# @markdown Embedding
EasyNegative = True # @param {type:"boolean"}
other_embedding_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown Hypernetwork
hypernetwork_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown LoRA
korean_doll_likeness = True # @param {type:"boolean"}
taiwan_doll_likeness = True # @param {type:"boolean"}
japanese_doll_likeness = True # @param {type:"boolean"}
UPRM = True # @param {type:"boolean"}
ChilloutMix = False # @param {type:"boolean"}
aEros = True # @param {type:"boolean"}
HassanBlend = True # @param {type:"boolean"}
Clarity = True # @param {type:"boolean"}
breastInClass = True # @param {type:"boolean"}
POVBJer = True # @param {type:"boolean"}
Titfuck = True # @param {type:"boolean"}
lora_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown - [EasyNegative](https://civitai.com/models/7808/easynegative)
# @markdown   - common negative promts embedding
# @markdown   - Trigger word is "EasyNegative", by adding it to the negative prompt
# @markdown - doll_likeness series([Korea](https://civitai.com/models/7448/korean-doll-likeness), [Taiwan](https://civitai.com/models/7716/taiwan-doll-likeness), [Japan](https://civitai.com/models/10135/japanese-doll-likeness))
# @markdown   - Trigger promt is "woman", "girl"
# @markdown   - Recommand to set the model to 0.5
# @markdown - You may add more model to it by insert multiple URL, seperate it with ","
# @markdown - [Please add the LoRA preview picture on you own, the format is <model name>.png](https://i.imgur.com/XooSV8p.png)

embeddings = []
other_embedding_urls = list(map(str.strip, embedding_urls.split(",")))
if EasyNegative == True:
    embeddings.append("https://civitai.com/api/download/models/9208")
if other_embedding_urls != ['']:
    embeddings = embeddings + other_embedding_urls

if embeddings != []:
    %cd {models_path}/embeddings
    for embedding in embeddings:
        ! wget -nc --content-disposition {embedding}

hypernetworks = []
hypernetwork_urls = list(map(str.strip, hypernetwork_urls.split(",")))
if hypernetwork_urls != ['']:
    hypernetworks = hypernetworks + hypernetwork_urls

if hypernetworks != []:
    %cd {models_path}/hypernetworks
    for hypernetwork in hypernetworks:
        ! wget -nc --content-disposition {hypernetwork}

loras = []
lora_urls = list(map(str.strip, lora_urls.split(",")))
if korean_doll_likeness == True:
    loras.append("https://civitai.com/api/download/models/14014")
if taiwan_doll_likeness == True:
    loras.append("https://civitai.com/api/download/models/9070")
if japanese_doll_likeness == True:
    loras.append("https://civitai.com/api/download/models/12050")
if UPRM == True:
    loras.append("https://civitai.com/api/download/models/4971")
if ChilloutMix == True:
    loras.append("https://civitai.com/api/download/models/11745")
if aEros == True:
    loras.append("https://civitai.com/api/download/models/5180")
if HassanBlend == True:
    loras.append("https://civitai.com/api/download/models/4635")
if Clarity == True:
    loras.append("https://civitai.com/api/download/models/13128")
if breastInClass == True:
    loras.append("https://civitai.com/api/download/models/13300")
if POVBJer == True:
    loras.append("https://civitai.com/api/download/models/10690")
if Titfuck == True:
    loras.append("https://civitai.com/api/download/models/9042")  
if lora_urls != ['']:
    loras = loras + lora_urls

if loras != []:
    %cd {models_path}/Lora
    for lora in loras:
        ! wget -nc --content-disposition {lora}

NameError: ignored

## 3 - Advanced Configuration

In [ ]:
# @title 3.1 Commandline Args.
import os

os.environ["COMMANDLINE_ARGS"] = "--no-half-vae --xformers --share --gradio-queue --hide-ui-dir-config --enable-insecure-extension-access --theme dark"

In [ ]:
# @title 3.2 Extensions
tag_complete = True # @param {type:"boolean"}
image_browser = True # @param {type:"boolean"}
control_net = False # @param {type:"boolean"}
openpose_editor = False # @param {type:"boolean"}
extension_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown - tag_complete provided "Danbooru tag" recommandation
# @markdown - image_browser will provide picture preview
# @markdown - control_net provide pose control
# @markdown - openpose_editor will able to build the pose with openpose method
# @markdown - Seperate the url with "," if you need to import more extension for use<br>

extensions = []
extension_urls = list(map(str.strip, extension_urls.split(",")))
if tag_complete == True:
    extensions.append("https://github.com/DominikDoom/a1111-sd-webui-tagcomplete")
if image_browser == True:
    extensions.append("https://github.com/AlUlkesh/stable-diffusion-webui-images-browser")
if control_net == True:
    extensions.append("https://github.com/Mikubill/sd-webui-controlnet")
if openpose_editor == True:
    extensions.append("https://github.com/fkunn1326/openpose-editor")
if extension_urls != ['']:
    extensions = extensions + extension_urls

import os

if extensions != []:
    %cd /content/stable-diffusion-webui/extensions
    for extension in extensions:
        if extension.startswith("#"):
            continue
        ! git clone {extension}
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.isdir(extension_name):
            ! git clone {extension}
        else:
            ! cd {extension_name} && git fetch

In [ ]:
# @title 3.3 ControlNet model import
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = True # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}
# @markdown ---
# @markdown - Reference :
# @markdown  - [Stable diffusion ControlNet使用心得](https://home.gamer.com.tw/artwork.php?sn=5662905)(Chinese)
# @markdown  - [姿勢生成網站](https://avatarposemaker.deezein.com/)(Chinese)
# @markdown  - https://github.com/lllyasviel/ControlNet
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
import os

control_net_models = []
if control_canny == True:
    control_net_models.append("control_canny")
if control_depth == True:
    control_net_models.append("control_depth")
if control_hed == True:
    control_net_models.append("control_hed")
if control_mlsd == True:
    control_net_models.append("control_mlsd")
if control_normal == True:
    control_net_models.append("control_normal")
if control_openpose == True:
    control_net_models.append("control_openpose")
if control_scribble == True:
    control_net_models.append("control_scribble")
if control_seg == True:
    control_net_models.append("control_seg")
if t2iadapter_keypose == True:
    control_net_models.append("t2iadapter_keypose")
if t2iadapter_seg == True:
    control_net_models.append("t2iadapter_seg")
if t2iadapter_sketch == True:
    control_net_models.append("t2iadapter_sketch")

%cd /content/stable-diffusion-webui/extensions

if os.path.exists('sd-webui-controlnet'):
    %cd sd-webui-controlnet/models
    for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/{control_net_model}-fp16.safetensors

## 4 - Build envoriment

In [ ]:
%cd /content

conda_dir = "/opt/conda"
conda_bin = os.path.join(conda_dir, "bin", "conda")

if not os.path.exists(conda_bin):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p {conda_dir}
    ! rm Miniconda3-latest-Linux-x86_64.sh


install_script = f"""
eval "$({conda_bin} shell.bash hook)"
cd stable-diffusion-webui
python3 -m pip install --upgrade tensorrt triton
python -c 'from launch import prepare_environment; prepare_environment()'
pip install --upgrade fastapi==0.90.1
"""

! {install_script}

patches_dir = "/content/patches"
os.makedirs(patches_dir, exist_ok=True)
! cd {patches_dir} && curl -LO https://raw.githubusercontent.com/ddPn08/automatic1111-colab/main/patches/stablediffusion-lowram.patch
! cd /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && git apply {patches_dir}/stablediffusion-lowram.patch

os.environ[
    "LD_LIBRARY_PATH"
] = f"{os.environ['LD_LIBRARY_PATH']}:/usr/local/envs/automatic/lib"

if os.path.exists(f"{data_dir}/script.post.sh"):
    ! chmod +x {data_dir}/script.post.sh
    ! {data_dir}/script.post.sh

## 5 - Startup WebUI

In [ ]:
# @markdown - Open the WebUI by click on the public URL<br>
# @markdown ![](https://i.imgur.com/y3xGiIX.png)

%cd /content/stable-diffusion-webui/
! git pull

run_script = f"""
eval "$({conda_bin} shell.bash hook)"
accelerate launch --num_cpu_threads_per_process 1 launch.py
"""
! {run_script}

## 6 - Save the styles.csv

In [ ]:
! cp /content/stable-diffusion-webui/styles.csv {config_path}/styles.csv
! cp /content/stable-diffusion-webui/styles.csv.bak {config_path}/styles.csv.bak